# Discription

In this data i have classified 4 types of roads which are good, poor, satisfactory and very poor using CNN. Upvote if you like my work.
Now lets get Started:

# First i am going to split data in x and y.

In [173]:
import os
import cv2

labels = ['good', 'poor', 'satisfactory', 'very_poor']

data_dir =  "data/sih_road_dataset/"

x = []
y = []
for label in labels:
    data = os.path.join(data_dir,label)
    for image in os.listdir(data):
        try:
            im = cv2.imread(os.path.join(data,image),cv2.IMREAD_COLOR)
            im = cv2.resize(im,(224,224))
            # Using the Canny filter with different parameters

            x.append(im)
            y.append(labels.index(label))


        except Exception as e:
            pass

In [174]:
# Just checking the legth of x and y
print(len(x))        
print(len(y))

1422
1422


# Changing list to array and rescaling the Data

In [175]:
import numpy as np
np.unique(y)
np.unique(x)

x = np.array(x)/255.0
y = np.array(y)

x.shape
y.shape

x = x.reshape(-1, 224, 224, 3)
x.shape
y = y.reshape(-1, 1)

# Turning y labels to categorical and Analysing Images

In [176]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y,4,)

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagenerator = ImageDataGenerator(
fill_mode= 'nearest',
horizontal_flip=False,
vertical_flip=False,
shear_range=0.1,
zoom_range = 0.1, # Randomly zoom image 
width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
height_shift_range=0.2
)
datagenerator.fit(x)

# Splitting Data into train and test

In [177]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y)

# Using CNN (Convolutional neural Network)

In [178]:
import tensorflow as tf
model = tf.keras.Sequential([
tf.keras.layers.Conv2D(32,(5,5),padding ='same',strides=(2,2),activation='relu',input_shape=(224,224,3)),
tf.keras.layers.MaxPooling2D((2,2)),
tf.keras.layers.Conv2D(64,(5,5),padding ='same',strides=(2,2),activation='relu'),
tf.keras.layers.MaxPooling2D((2,2)),
tf.keras.layers.Conv2D(128,(5,5),padding ='same',strides=(2,2),activation='relu'),
tf.keras.layers.MaxPooling2D((2,2)),
tf.keras.layers.Conv2D(128,(5,5),padding ='same',strides=(2,2),activation='relu'),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(256,activation='relu'),
tf.keras.layers.Dropout(0.2),
    
tf.keras.layers.Dense(4,activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# Using early stopping and training the model

#### Note: EarlyStopping stop the model from overfitting.

In [215]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

model.fit(datagenerator.flow(x_train,y_train,batch_size=28),epochs=20,callbacks=[earlystop],validation_data=datagenerator.flow(x_test,y_test))

Epoch 1/20
39/39 [==============================] - 14s 352ms/step - loss: 0.2654 - accuracy: 0.8940 - val_loss: 0.3340 - val_accuracy: 0.8652
Epoch 2/20
39/39 [==============================] - 20s 527ms/step - loss: 0.2960 - accuracy: 0.8902 - val_loss: 0.4134 - val_accuracy: 0.8427
Epoch 3/20
39/39 [==============================] - 17s 423ms/step - loss: 0.2843 - accuracy: 0.8856 - val_loss: 0.4027 - val_accuracy: 0.8258
Epoch 4/20
39/39 [==============================] - 16s 419ms/step - loss: 0.3015 - accuracy: 0.8724 - val_loss: 0.3424 - val_accuracy: 0.8792
Epoch 5/20
39/39 [==============================] - 16s 423ms/step - loss: 0.2614 - accuracy: 0.8996 - val_loss: 0.3000 - val_accuracy: 0.8820
Epoch 6/20
39/39 [==============================] - 16s 417ms/step - loss: 0.2553 - accuracy: 0.8996 - val_loss: 0.3207 - val_accuracy: 0.8736
Epoch 7/20
39/39 [==============================] - 16s 414ms/step - loss: 0.2476 - accuracy: 0.9006 - val_loss: 0.3096 - val_accuracy: 0.9017

# Plotting loss|val_loss to see if model overfitting or not

#### Note: If val_loss is higher then loss it means model is overfitting.

In [203]:
import pandas as pd
history = pd.DataFrame(model.history.history)
history[["loss", "val_loss"]].plot()

KeyError: "None of [Index(['loss', 'val_loss'], dtype='object')] are in the [columns]"

# At last evaluating the model to check accuracy

In [216]:
model.evaluate(x_test, y_test)

12/12 [==============================] - 1s 50ms/step - loss: 0.3791 - accuracy: 0.8876


[0.3790539801120758, 0.8876404762268066]

# And here is the Prediction of road

In [181]:
from PIL import Image
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((224, 224))
    return np.array(image)

In [182]:
def get_profile_name(label):
    if label==0:
        return "Good"
    if label==1:
        return "Poor"
    if label==2:
        return "Satisfactory"
    if label==3:
        return "Very_Poor"

In [183]:
def predict_profile(file):
    print("Predicting .................................")
    ar=convert_to_array(file)
    ar=ar/255
    a=[]
    a.append(ar)
    a=np.array(a)
    score=model.predict(a,verbose=1)
    print(score)
    label_index=np.argmax(score)
    print(label_index)
    acc=np.max(score)
    profile=get_profile_name(label_index)
    print(profile)
    print("The predicted profile is a "+profile+" with accuracy = "+str(acc))  
    return profile

## Predicting Very poor road

In [184]:
pre_image = "data/sih_road_dataset/very_poor/verypoor_014.jpg"

In [185]:
predict_profile(pre_image)

Predicting .................................
1/1 [==============================] - 0s 87ms/step
[[9.2844537e-05 3.1308343e-03 4.1629067e-03 9.9261343e-01]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.99261343


'Very_Poor'

## Predicting Good road

In [154]:
pre_image2 = "data/sih_road_dataset/good/good_040.jpg"


In [155]:
predict_profile(pre_image2)

Predicting .................................


AttributeError: 'NoneType' object has no attribute '__array_interface__'

## Predicting Satisfactory road

Note: in this data poor roads and satisfactory road are very similar to each other thats why model wont be able to predict or differentiate between those two images.

# Test pictures

In [204]:
import os
base_path = os.getcwd() + '/kaggle'

In [217]:
file_list = [filename for filename in os.listdir(base_path + '/input/') if not filename.startswith('.')]
count = 0
lenn = len(file_list)
dict = {'Poor' : 0, 'Good' : 0, 'Satisfactory' : 0, 'Very_Poor' : 0}
for file in file_list:
    path = 'kaggle/input/'+str(file)
    print(file)
    
#     ind = str(file)[-5]
    label = predict_profile(path)
#     if (ind==str(label).lower()[0]):
#         count+=1
    print()
    dict[label] += 1

AA_10800.jpeg
Predicting .................................
1/1 [==============================] - 0s 13ms/step
[[0.15792327 0.1918445  0.46525317 0.18497899]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.46525317

AA_10810.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms/step
[[0.1316077  0.1750067  0.48738536 0.20600024]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.48738536

AB_1000.jpeg
Predicting .................................
1/1 [==============================] - 0s 17ms/step
[[0.07890265 0.13726565 0.42790097 0.35593078]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.42790097

AB_1010.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.0982977  0.15055062 0.5882055  0.16294618]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.5882055

AB_1020.jpeg
Predicting ...........

1/1 [==============================] - 0s 18ms/step
[[0.01439754 0.03408246 0.4260982  0.52542174]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.52542174

AC_15980.jpeg
Predicting .................................
1/1 [==============================] - 0s 21ms/step
[[0.06166143 0.09465142 0.3791094  0.4645778 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.4645778

AC_16000.jpeg
Predicting .................................
1/1 [==============================] - 0s 21ms/step
[[0.07937609 0.22913589 0.5856665  0.10582159]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.5856665

AC_16020.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms/step
[[0.05999594 0.3774     0.49091038 0.07169374]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.49091038

AC_16030.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms

1/1 [==============================] - 0s 10ms/step
[[0.04212789 0.07210893 0.6502086  0.23555464]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.6502086

AC_23040.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms/step
[[0.03704842 0.0718324  0.5029851  0.3881341 ]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.5029851

AC_23050.jpeg
Predicting .................................
1/1 [==============================] - 0s 19ms/step
[[0.03537909 0.08966338 0.44442132 0.43053618]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.44442132

AC_23060.jpeg
Predicting .................................
1/1 [==============================] - 0s 18ms/step
[[0.01374461 0.04970018 0.20309605 0.7334592 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.7334592

AC_23070.jpeg
Predicting .................................
1/1 [==============================] - 0s

1/1 [==============================] - 0s 20ms/step
[[0.00640098 0.3631294  0.6241692  0.00630047]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.6241692

AC_2970.jpeg
Predicting .................................
1/1 [==============================] - 0s 12ms/step
[[0.01629569 0.20195849 0.75238186 0.02936396]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.75238186

AC_2980.jpeg
Predicting .................................
1/1 [==============================] - 0s 10ms/step
[[0.00534056 0.5098732  0.47830284 0.00648348]]
1
Poor
The predicted profile is a Poor with accuracy = 0.5098732

AC_31150.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms/step
[[0.01294373 0.55441046 0.37555826 0.05708752]]
1
Poor
The predicted profile is a Poor with accuracy = 0.55441046

AC_31160.jpeg
Predicting .................................
1/1 [==============================] - 0s 10ms/step
[[0.01110422 0.0


AC_3980.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms/step
[[0.01459949 0.33651727 0.5831604  0.0657229 ]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.5831604

AC_3990.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms/step
[[0.01696777 0.39167225 0.5866949  0.00466511]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.5866949

AC_4000.jpeg
Predicting .................................
1/1 [==============================] - 0s 11ms/step
[[0.01619453 0.4601986  0.5204511  0.00315577]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.5204511

AC_4020.jpeg
Predicting .................................
1/1 [==============================] - 0s 21ms/step
[[0.01982585 0.37250564 0.5126345  0.09503393]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.5126345

AC_410.jpeg
Predicting ................

...

1/1 [==============================] - 0s 24ms/step
[[0.01955025 0.09985954 0.09489945 0.78569067]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.78569067

Y_44770.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.03067239 0.12931733 0.10092738 0.739083  ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.739083

Y_44780.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.02212673 0.10679472 0.08630571 0.7847728 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.7847728

Y_44790.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.03881138 0.15401553 0.12693249 0.68024063]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.68024063

Y_44800.jpeg
Predicting .................................
1/1 [==============================] - 0s 20ms/step
[[0.0871565

1/1 [==============================] - 0s 16ms/step
[[0.01054105 0.06644934 0.11201243 0.8109971 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.8109971

Y_47130.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.07246836 0.23722765 0.1543755  0.5359285 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.5359285

Y_47140.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.04524161 0.17644984 0.1436407  0.6346679 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.6346679

Y_47150.jpeg
Predicting .................................
1/1 [==============================] - 0s 24ms/step
[[0.05147033 0.18821357 0.16880877 0.5915073 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.5915073

Y_47160.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.02723825

1/1 [==============================] - 0s 16ms/step
[[0.06585936 0.18423001 0.26645857 0.48345208]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.48345208

Y_48320.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.02681698 0.13095799 0.16029748 0.6819275 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.6819275

Y_48340.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.01751799 0.08485823 0.16921525 0.7284086 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.7284086

Y_48350.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.02052512 0.09137679 0.1969091  0.6911891 ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.6911891

Y_48380.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.0011052

1/1 [==============================] - 0s 16ms/step
[[0.11118022 0.18101509 0.31476513 0.39303952]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.39303952

Y_50190.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.11895464 0.18474619 0.30894837 0.38735083]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.38735083

Y_50200.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.1386581  0.17307572 0.3322619  0.35600433]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.35600433

Y_50230.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.1852644  0.14325097 0.35370624 0.3177784 ]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.35370624

Y_50240.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[

1/1 [==============================] - 0s 16ms/step
[[0.06528927 0.20732944 0.40984234 0.31753895]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.40984234

Z_3000.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.06734822 0.14274463 0.4217723  0.36813492]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.4217723

Z_3020.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.07933543 0.13308793 0.45354345 0.33403322]]
2
Satisfactory
The predicted profile is a Satisfactory with accuracy = 0.45354345

Z_3040.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/step
[[0.04915244 0.08875804 0.40770045 0.454389  ]]
3
Very_Poor
The predicted profile is a Very_Poor with accuracy = 0.454389

Z_430.jpeg
Predicting .................................
1/1 [==============================] - 0s 16ms/ste

In [218]:
for elem in dict:
    print(elem,dict[elem],  dict[elem]/lenn*100)

Poor 417 12.950310559006212
Good 268 8.322981366459627
Satisfactory 1215 37.732919254658384
Very_Poor 1320 40.993788819875775
